### 공급량 데이터 수집 및 저장 (특정 연도와 월 지정)

In [69]:
import pandas as pd

path = 'D:/PythonProject/data-gatherer/dse/data/2. 천연가스공급량(일별)_2012년_2023년6월_230704.xlsx'
start_row = 2 #시작하는 열
end_row = 31 #끝나는 열


In [70]:
# 특정 연도와 월의 해당일자(1일 ~ 말일)의 공급량 데이터 추출
def read_data(path, sheet, year, month):
    use_cols= [0, month]    # 6월 데이터: 첫번째 열('구 분') 및 6번째 열
    df = pd.read_excel(path, sheet_name=sheet, header=start_row, usecols=use_cols, nrows=end_row)
    df = df.set_index('구 분') #인덱스를 '구분'셀로 변경
    df = df.fillna(-1).astype('int').replace({-1: None}) #정수형으로 데이터 변환
    new_df = pd.DataFrame(columns=['year','month','day','supply']) #임의의 데이터프레임 생성 >> 데이터를 전처리하여 넣을 곳
    for i in df.columns:
        for j in df.index:  #j는 일(day)
            supply_value = df.loc[j][i]
            #값이 있는 경우 및 값이 0보다 큰 경우만 해당됨 
            if isinstance(supply_value, int) and supply_value > 0:
                array=[] #임시 배열 추가(값이 계속 초기화됨)
                array.extend([year, i, j, supply_value]) #데이터프레임에 값을 추가하기 위해 형식 변경
                result1 = "".join(map(str, array)) #고유값 생성(인덱스 사용을 위해서)
                new_df.loc[result1] = array #고유값의 인덱스에 값을 넣어줌
    new_df = new_df.reset_index(drop=True) #인덱스 초기화
    new_df['date'] = pd.to_datetime(new_df[['year', 'month', 'day']], format = '%Y%M%D')
    new_df = new_df.drop(labels=['year','month','day'],axis=1) #임시로 생성한 날짜 형식의 데이터 제거
    new_df['date_id'] = new_df['date'].dt.strftime('%Y%m%d') #데이터 형식 변경
    new_df = new_df.drop(labels=['date'],axis=1) #임시로 생성한 날짜 형식의 데이터 제거
    new_df = new_df.reindex(['date_id', 'supply'], axis = 1)
    return new_df

In [71]:
df_supply_m3 = read_data(path,'2023_부피', 2023, 6)
df_supply_m3.rename(columns={'supply':'supply_m3'},inplace=True)
df_supply_mj = read_data(path,'2023_열량', 2023, 6)
df_supply_mj.rename(columns={'supply':'supply_mj'},inplace=True)

In [77]:
df_supply_mj.shape, df_supply_m3.shape

((30, 2), (30, 2))

In [74]:
# 데이터 결합
df_supply = pd.merge(df_supply_mj, df_supply_m3, left_on='date_id', right_on='date_id', how='outer')
df_supply

,date_id,supply_mj,supply_m3
0,20230601,80271126,1887432
1,20230602,78088792,1837501
2,20230603,60904197,1434642
3,20230604,53719220,1264695
4,20230605,67448344,1584963
5,20230606,63783350,1499277
6,20230607,75074358,1765133
7,20230608,75545097,1777401
8,20230609,73343119,1722373
9,20230610,58065276,1364050


In [ ]:
# print("중복 제거 전 데이터 길이 : ",len(df_supply))
# # 중복거를 통한 데이터 길이 확인
# df_supply.drop_duplicates(subset=None, keep='last', inplace=True, ignore_index=False) #가장 마지막 데이터값을 값으로 남겨둠
# print("중복 제거 후 데이터 길이 : ",len(df_supply))

In [75]:
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_supply.itertuples():
    sql = "insert into supply(date_id, supply_mj, supply_m3) values (%s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3]))

conn.commit()
conn.close()